# AOB MC Data Exploration
The goal of this notebook is to take in data from Zhuang et al. (see other notebooks) and be able to explore spatial transcriptomics of AOB MCs, specifically. This will be done in a few steps including:
* data loading
* joining dataframes
* filtering for excitatory neurons within the accessory olfactory bulb
* visualizing genetic diversity of these neurons

In [1]:
# if using for the first time, you will need to download the Allen Brain-specific functions:
!pip install "abc_atlas_access[notebooks] @ git+https://github.com/alleninstitute/abc_atlas_access.git"

  Cloning https://github.com/alleninstitute/abc_atlas_access.git to c:\users\kevjm\appdata\local\temp\pip-install-bgyyz6pe\abc-atlas-access_2ac1866e72c04a7381156c9c1d63f569
  Resolved https://github.com/alleninstitute/abc_atlas_access.git to commit 398c9cc557a4b1e75dc68dd3d4965d3ed50367b2
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/785.6 kB ? eta -:--:--
   --------------------------------------- 785.6/785.6 kB 14.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.9 MB ? eta -:--:--
   ------------ --------------------------- 4.2/13.9 MB 19.9 MB/s eta 0:00:01
   -------------------- ------------------- 7.1/13.9 MB 18.5 MB/s eta 0:00:01
   

  Running command git clone --filter=blob:none --quiet https://github.com/alleninstitute/abc_atlas_access.git 'C:\Users\kevjm\AppData\Local\Temp\pip-install-bgyyz6pe\abc-atlas-access_2ac1866e72c04a7381156c9c1d63f569'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.19.0 requires botocore<1.36.4,>=1.36.0, but you have botocore 1.39.13 which is incompatible.


## Imports and loading of relevant data

In [3]:
# import relevant libraries and Allen-specific packages to engage with datasets
import pandas as pd
from pathlib import Path
import numpy as np
import anndata
import time
import matplotlib.pyplot as plt

from abc_atlas_access.abc_atlas_cache.abc_project_cache import AbcProjectCache

In [2]:
# defining heatmap visualization
def plot_heatmap(df, fig_width = 8, fig_height = 4, cmap=plt.cm.magma_r, vmax=None):

    arr = df.to_numpy()

    fig, ax = plt.subplots()
    fig.set_size_inches(fig_width, fig_height)

    res = ax.imshow(arr, cmap=cmap, aspect='auto', vmax=vmax)
    xlabs = df.columns.values
    ylabs = df.index.values

    ax.set_xticks(range(len(xlabs)))
    ax.set_xticklabels(xlabs)

    ax.set_yticks(range(len(ylabs)))
    res = ax.set_yticklabels(ylabs)

In [4]:
# loading first 3 datasets (4th does not have AOB neurons)
datasets = ['Zhuang-ABCA-1', 'Zhuang-ABCA-2', 'Zhuang-ABCA-3']
# loading in cell metadata for each dataframe (a dataframe given by the datasets above)
cell = {}
for d in datasets :

    cell[d] = abc_.get_metadata_dataframe(
        directory=d,
        file_name='cell_metadata',
        dtype={"cell_label": str}
    )
    cell[d].set_index('cell_label', inplace=True)
    
    sdf = cell[d].groupby('brain_section_label')
    
    print(d,":","Number of cells = ", len(cell[d]), ", ", "Number of sections =", len(sdf))

TypeError: AbcProjectCache.get_metadata_dataframe() missing 1 required positional argument: 'self'